In [ ]:
# ========================================
# STEP 1. INSTALL DEPENDENCIES
# ========================================
!pip install --upgrade --no-cache-dir torch torchvision torchaudio
!pip install --upgrade --no-cache-dir unsloth unsloth_zoo trl datasets huggingface_hub pandas

# Sau khi cài xong, RESTART runtime để tránh import vòng lặp
import os
os.kill(os.getpid(), 9)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 177.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 222.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 192.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 143.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 167.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 272.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# ========================================
# STEP 2. IMPORT LIBRARIES
# ========================================
import unsloth
import os, json, torch, pandas as pd

from datasets import Dataset, DatasetDict, load_dataset
from huggingface_hub import notebook_login
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer

print("""
========================================
     IMPORT LIBRARIES SUCCESSFULLY
========================================
""")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!

     IMPORT LIBRARIES SUCCESSFULLY



In [6]:
# ========================================
# STEP 3. LOGIN TO HUGGINGFACE
# ========================================
notebook_login()


In [9]:
# ========================================
# STEP 4. PREPARE DATASET (Load from HF)
# ========================================
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

# Hugging Face repo
huggingface_user = "PLKhang"
dataset_name = "report-finetuning-prompts"


# Load dataset từ Hugging Face Hub
llama_dataset = load_dataset(
    f"{huggingface_user}/{dataset_name}",
    split="train",
)

print("✅ Dataset ready for fine-tuning!")
print(llama_dataset)
print(llama_dataset[0]["prompt"][:500])



✅ Dataset ready for fine-tuning!
Dataset({
    features: ['prompt'],
    num_rows: 1797
})
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a team leader writing internal daily progress notes.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Generate DAILY_PROGRESS_NOTE based on the following project context:

Project: HealthTrack Mobile App (A secure mobile application that allows patients to monitor vital signs, schedule appointments, and access electronic health records.)
Phase: Project Initiation & Requirements Gathering (Define project scope, objectives, an


In [ ]:
# ========================================
# STEP 5. CONFIGURATION
# ========================================
config = {
    "model_config": {
        "base_model": "unsloth/llama-3-8b-Instruct-bnb-4bit",
        "finetuned_model_gguf": "llama-3-8b-report-gen-v2",
        "finetuned_model_standard": "llama-3-8b-report-gen-v3",
        "max_seq_length": 1024,
        "dtype": torch.float16,
        "load_in_4bit": True,
    },
    "lora_config": {
        "r": 8,
        "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                           "gate_proj", "up_proj", "down_proj"],
        "lora_alpha": 16,
        "lora_dropout": 0.05,
        "bias": "none",
        "use_gradient_checkpointing": True,
    },
    "training": {
        "batch_size": 1,
        "gradient_accumulation_steps": 4,
        "num_train_epochs": 3,
        "learning_rate": 2e-4,
        "logging_steps": 5,
        "output_dir": "outputs",
    },
    "dataset": {
        "name": f"{huggingface_user}/{dataset_name}",
        "split": "train",
        "text_field": "prompt",
    }
}


In [12]:
# ========================================
# STEP 6. LOAD BASE MODEL + LORA SETUP
# ========================================
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = config["model_config"]["base_model"],
    max_seq_length = config["model_config"]["max_seq_length"],
    dtype = config["model_config"]["dtype"],
    load_in_4bit = config["model_config"]["load_in_4bit"],
)

model = FastLanguageModel.get_peft_model(
    model,
    r = config["lora_config"]["r"],
    target_modules = config["lora_config"]["target_modules"],
    lora_alpha = config["lora_config"]["lora_alpha"],
    lora_dropout = config["lora_config"]["lora_dropout"],
    bias = config["lora_config"]["bias"],
    use_gradient_checkpointing = config["lora_config"]["use_gradient_checkpointing"],
)


==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.5 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [9]:
# ========================================
# STEP 7. LOAD DATASET
# ========================================
dataset_train = load_dataset(config["dataset"]["name"], split=config["dataset"]["split"])


In [11]:
# ========================================
# STEP 8. TRAINER SETUP
# ========================================

def formatting_func(batch):
    texts = []
    for prompt in batch["prompt"]:
        texts.append(prompt)
    return texts

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    formatting_func=formatting_func,
    max_seq_length=config["model_config"]["max_seq_length"],
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=config["training"]["batch_size"],
        gradient_accumulation_steps=config["training"]["gradient_accumulation_steps"],
        num_train_epochs=config["training"]["num_train_epochs"],
        learning_rate=config["training"]["learning_rate"],
        fp16=True,
        logging_steps=config["training"]["logging_steps"],
        output_dir=config["training"]["output_dir"],
        optim="adamw_8bit",
        lr_scheduler_type="linear",
        seed=42,
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1797 [00:00<?, ? examples/s]

In [12]:
# ========================================
# STEP 9. TRAIN MODEL
# ========================================
trainer_stats = trainer.train()
with open("trainer_stats.json", "w") as f:
    json.dump(trainer_stats, f, indent=4)


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,797 | Num Epochs = 3 | Total steps = 1,350
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 20,971,520 of 8,051,232,768 (0.26% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: plkhang090403 (plkhang090403_n21dccn042) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,2.542800
10,1.518100
15,1.056400
20,0.961100
25,0.924300
30,0.775700
35,0.710800
40,0.650800
45,0.649200
50,0.675700


train/epoch,▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██████
train/grad_norm,██▆▆▅▄▄▃▃▃▃▂▃▂▂▃▂▂▂▃▃▃▃▂▃▁▂▂▂▂▁▂▂▂▁▂▁▃▂▂
train/learning_rate,████▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▂▂▂▂▂▂▁▁▁▁
train/loss,█▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,6.457264088113152e+16
train/epoch,3
train/global_step,1350
train/grad_norm,1.05036
train/learning_rate,0.0
train/loss,0.136


In [ ]:
# OOM: LOAD FROM LAST CHECKPOINT
# Select checkpoint-1350.
checkpoint_path = "/content/outputs/checkpoint-1350"
model.load_adapter(checkpoint_path, adapter_name="final_checkpoint")

print(f"Đã load các trọng số đã train từ {checkpoint_path} thành công!")

Đã load các trọng số đã train từ /content/outputs/checkpoint-1350 thành công!


In [ ]:
# ========================================
# STEP 10. PUSH MODEL (GGUF ONLY)
# ========================================
# finetuned_model_name = config["model_config"]["finetuned_model_gguf"]

# model.push_to_hub_gguf(
#     finetuned_model_name,
#     tokenizer,
#     quantization_method = "q4_k_m"
# )

# Cause of OOM -> reset kernel

NameError: name 'config' is not defined

In [16]:
# ========================================
# STEP 10. PUSH MODEL (STANDARD HF FORMAT)
# ========================================
finetuned_model_name = config["model_config"]["finetuned_model_standard"]
huggingface_repo = f"{huggingface_user}/{finetuned_model_name}"

# 1. Đẩy LoRA adapters và cấu hình mô hình
# đẩy tất cả các file cần thiết (adapter_model.bin, config.json, ..) lên Hugging Face Hub.
model.push_to_hub(huggingface_repo, token=True)

# 2. Đẩy tokenizer
tokenizer.push_to_hub(huggingface_repo, token=True)

print(f"Mô hình đã được đẩy lên: https://huggingface.co/{huggingface_repo}")

README.md:   0%|          | 0.00/592 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   7%|6         | 5.56MB / 83.9MB            

  ...adapter_model.safetensors:   1%|          |  557kB / 83.9MB            

Saved model to https://huggingface.co/PLKhang/llama-3-8b-report-gen-v3


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpn1bff7o9/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

Mô hình đã được đẩy lên: https://huggingface.co/PLKhang/llama-3-8b-report-gen-v3


In [18]:
# ========================================
# STEP 11. TEST MODEL
# ========================================
FastLanguageModel.for_inference(model)

test_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a software engineer reporting progress to your manager.
<|eot_id|><|start_header_id|>user<|end_header_id|>
Generate IN_PROGRESS_EMAIL.

Project: Website Revamp
Phase: Frontend
Task: Implement login feature
Subtask: Validate input fields
Status: in_progress
<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 250,
    temperature = 0.2,
    do_sample = False,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a software engineer reporting progress to your manager.
<|eot_id|><|start_header_id|>user<|end_header_id|>
Generate IN_PROGRESS_EMAIL.

Project: Website Revamp
Phase: Frontend
Task: Implement login feature
Subtask: Validate input fields
Status: in_progress
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Subject: Update on Website Revamp: Frontend - Implement Login Feature

Dear [Manager's Name],

I hope this email finds you well. I am writing to provide an update on the progress of the Website Revamp project, specifically on the frontend phase.

As previously discussed, the task at hand is to implement the login feature. I am pleased to report that I have made significant progress on the subtask of validating input fields.

Currently, I am working on implementing the following:

* Validating the username field to ensure it meets the required format and length
* Validating the password field to